In [ ]:
# I am just practicing things based on the following sources:
# 
# Video Tutorial: https://www.youtube.com/watch?v=VMj-3S1tku0
# GitHub repo: https://github.com/karpathy/micrograd

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class Value:
    
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self._prev = set(_children)
        self.label = label
        self._op = _op
        self.grad = 0
        
    def __add__(self, other):
        return Value(self.data + other.data, _children=(self, other), _op='+')
    
    def __mul__(self, other):
        return Value(self.data * other.data, _children=(self, other), _op='*')
        
    def __repr__(self):
        return f'Value(label={self.label}, data={self.data})'

In [ ]:
a = Value(2.0, label='a'); print(a)
b = Value(-3.0, label='b'); print(b)
c = a * b; c.label='c'; print(c)
d = Value(2.0, label='d'); print(d)
e = c + d; e.label='e'; print(e)
f = Value(2.0, label='f'); print(f)
L = e * f; L.label='L'; print(L)

In [ ]:
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
                
    build(root)
    return nodes,edges
    
def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
    
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid, label = "%s | data %.4f | grad %.4f" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name = uid + n._op, label= f"{n._op}")
            
            dot.edge(uid + n._op, uid)
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
            
    return dot
    

In [ ]:
draw_dot(L)

In [ ]:
h = 0.01

In [ ]:
def fn(a,b,d,f):
    c = a * b
    e = c + d
    L = e * f
    return L.data

In [ ]:
dL_dL = ((fn(a,b,d,f) + h) - fn(a,b,d,f))/h
L.grad = dL_dL

In [ ]:
f.grad = e.data
e.grad = f.data

d.grad = e.grad * L.grad
c.grad = e.grad * L.grad

b.grad = c.grad * a.data
a.grad = c.grad * b.data

In [ ]:
a.data += 0.01 * a.grad
b.data += 0.01 * b.grad
d.data += 0.01 * d.grad
f.data += 0.01 * f.grad

In [ ]:
c = a * b; c.label='c'
e = c + d; e.label='e'; print(e)
L = e * f; L.label='L'; print(L)

In [ ]:
draw_dot(L)

In [ ]:
class Value_Full:
    
    def __init__(self, data, label="", _children=(), _op=None):
        self.data = data
        self.grad = 0
        self.label = label
        self._prev = set(_children)
        self._op = _op
        self._backprop = lambda: None
    
    def __add__(self, other):
        data = other.data if isinstance(other, Value_Full) else other
        out = Value_Full(self.data + data, _children=(self, other), _op="+")

        def _add_backprop():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        
        self._backprop = _add_backprop
        return out

    def __mul__(self, other, _op="*"):
        data = other.data if isinstance(other, Value_Full) else other
        out = Value_Full(self.data * data, _children=(self, other), _op="*")
        out.grad += self.data * other.grad
        
        def _mul_backprop():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        self._backprop = _mul_backprop
        return out
    
    def reset_grads(self):
        for n in self._prev:
            n.reset_grads()
            n.grad = 0
        
    def backprop(self):
        
        self.reset_grads()
        
        def go_back(node):
            chain = []
            for n in node._prev:
                chain += go_back(n)
            chain.append(node)
            return chain
        
        self.grad = 1.0
        for n in reversed(go_back(self)):
            n._backprop()
        
    
    def __repr__(self):
        return f'Value_Full(label={self.label}, data={self.data}, grad={self.grad})'

In [ ]:

a = Value_Full(2, "a")
b = Value_Full(3, "b")
c = a + b; c.label="c"
d = a * b; d.label="d"
e = (c * d) + (a * b); e.label="e"

draw_dot(e)

In [ ]:
e.backprop()

In [ ]:
draw_dot(e)

In [ ]:
b.data += 0.01 * b.grad

c = a + b; c.label="c"
d = a * b; d.label="d"
e = (c * d) + (a * b); e.label="e"

e.backprop()
draw_dot(e)